In [1]:
from pypdf import PdfReader

In [6]:
pdf_path = r"C:\Users\Ruthvik\gov chat support system\dataset\Indian Constitution.pdf"

In [11]:
extract = PdfReader(pdf_path)

In [15]:
text = ""
for page in extract.pages:
    text = text + page.extract_text() or ''

852500